In [1]:
import pygame
import os
import time
import random
pygame.init()
pygame.font.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:


WIDTH,HEIGHT = 600,600
WIN = pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("Space Invaders")

# Attacking Ships and Lasers
RED_SPACESHIP = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\enemy.png")
GREEN_SPACESHIP = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\enemy.png")
BLUE_SPACESHIP = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\enemy.png")

RED_LASER = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\bullet.png")
GREEN_LASER = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\bullet.png")
BLUE_LASER = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\bullet.png")


# Player Ship and Lasers
YELLOW_SPACESHIP = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\pixel_ship_yellow.png")
YELLOW_LASER = pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\pixel_laser_yellow.png")

BG = pygame.transform.scale(pygame.image.load(r"C:\Data\Working folder\Notebook\Space Invaders\assets\background.jpg"),(WIDTH,HEIGHT))

class Laser:
    def __init__(self, x,y , img):
        self.x = x 
        self.y = y
        self.img = img
        self.mask = pygame.mask.from_surface(self.img)
    
    def draw(self,window):
        window.blit(self.img, (self.x , self.y))

    
    def move(self,vel):
        self.y += vel
        
    def off_screen(self,height):
        return not(self.y <= height and self.y >=0 )
    
    def collision(self , obj):
        return collide(self ,obj)


class Ship:
    COOLDOWN = 20
    
    def __init__(self,x,y,health=100):
        self.x = x
        self.y = y
        self.health = health
        self.ship_img = None
        self.laser_img = None
        self.lasers = []
        self.cool_down_counter = 0
    
    def draw(self,window):
        window.blit(self.ship_img , (self.x , self.y))
        for laser in self.lasers :
            laser.draw(window)
            
    def move_lasers(self,vel,obj):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            elif laser.collision(obj) :
                obj.health -= 10
                self.lasers.remove(laser)
    
    def get_width(self):
        return self.ship_img.get_width()
    
    def get_height(self):
        return self.ship_img.get_height()
    
    def shoot(self):
        if self.cool_down_counter == 0 :
            laser = Laser(self.x+35,self.y, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1
            
    def cooldown(self):
        if self.cool_down_counter >= self.COOLDOWN :
            self.cool_down_counter = 0
        elif self.cool_down_counter > 0:
            self.cool_down_counter += 1
            
class Player(Ship):
    
    def __init__(self ,x, y , health=100):
        super().__init__(x,y,health)
        self.ship_img = YELLOW_SPACESHIP
        self.laser_img = YELLOW_LASER
        self.mask = pygame.mask.from_surface(self.ship_img)
        self.max_health = health
        
    def move_lasers(self,vel,objs):
        self.cooldown()
        for laser in self.lasers:
            laser.move(vel)
            if laser.off_screen(HEIGHT):
                self.lasers.remove(laser)
            else:
                for obj in objs:
                    if laser.collision(obj):
                        objs.remove(obj)
                        
                        self.lasers.remove(laser)
      
    def healthbar(self,window):
        pygame.draw.rect(window,(255,0,0),(self.x, self.y+self.ship_img.get_height()+10, self.ship_img.get_width() ,10), 1)
        pygame.draw.rect(window,(0,255,0),
                 (self.x,int(self.y+self.ship_img.get_height()+10),int(self.ship_img.get_width()*(self.health/self.max_health)),10),1)

        
    def draw(self,window):
        super().draw(window)
        self.healthbar(window)
        
class Enemy(Ship):
    COLOR_MAP={"red":(RED_SPACESHIP,RED_LASER),"green":(GREEN_SPACESHIP,GREEN_LASER),"blue":(BLUE_SPACESHIP,BLUE_LASER)}
    def __init__(self,x,y,color,health=100):
        super().__init__(x,y,health)
        self.ship_img , self.laser_img = self.COLOR_MAP[color]
        self.mask = pygame.mask.from_surface(self.ship_img)
        
    def move(self,vel):
        self.y += vel
        
    def shoot(self):
        if self.cool_down_counter == 0 :
            laser = Laser(self.x+15,self.y, self.laser_img)
            self.lasers.append(laser)
            self.cool_down_counter = 1
    
def collide(obj1 , obj2):
    offset_x = obj2.x - obj1.x
    offset_y = obj2.y - obj1.y
    return  obj1.mask.overlap(obj2.mask, (offset_x , offset_y)) != None 
    
def main():
    score = 0
    run = True
    FPS = 60
    level = 0
    lives = 5
    clock = pygame.time.Clock() 
    main_font = pygame.font.SysFont("comicsans",30)
    lost_font = pygame.font.SysFont("comicsans",60)
    laser_vel = 4
    enemies = []
    wave_length = 5
    enemy_vel = 1
    
    player_vel = 5
    player =Player(0,400)
    lost = False
    lost_count = 0
    
    def redraw_window():
        WIN.blit(BG,(0,0))
        lives_label = main_font.render(f"Lives : {lives}",1,(255,255,255))
        level_label = main_font.render(f"Level : {level}",2,(255,255,255))
        WIN.blit(lives_label, (10,10))
        WIN.blit(level_label, (int(WIDTH - level_label.get_width() - 10),10))
        
        for enemy in enemies :
            enemy.draw(WIN)
        
        player.draw(WIN)
        
        if lost:
            lost_label = lost_font.render("GAME OVER",1, (255,255,255))
            WIN.blit(lost_label ,(int(WIDTH/2 - lost_label.get_width()/2) , 300))
        
        pygame.display.update()
    
    while run :
        redraw_window()
        
        clock.tick(FPS)
        
        if lives <= 0 or player.health <= 0 :
            lost = True
            lost_count+=1
        
        if lost:
            if lost_count > FPS * 3 :
                run = False
            else:
                continue
        
        if len(enemies)==0 :
            level +=1
            wave_length +=5
            for i in range(wave_length):
                enemy = Enemy(random.randrange(50,WIDTH-100) , random.randrange(-1500,-100) , random.choice(["red","blue","green"]))
                enemies.append(enemy)
                
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and player.x > 0 :
            player.x -= player_vel
        if keys[pygame.K_RIGHT] and player.x + player_vel + player.get_width() - 5 < WIDTH:
            player.x += player_vel
        if keys[pygame.K_UP] and player.y > 0 : 
            player.y -= player_vel
        if keys[pygame.K_DOWN] and player.y + player_vel + player.get_height() - 5 + 20 < HEIGHT :
            player.y += player_vel
        if keys[pygame.K_SPACE] :
            player.shoot()
        
        
        for enemy in enemies[:]:
            enemy.move(enemy_vel)
            enemy.move_lasers(laser_vel , player)
            
            if random.randrange(0,2*60) == 1:
                enemy.shoot()

            if collide(enemy,player):
                player.health -= 10
                enemies.remove(enemy)
                score+=1
                
            if enemy.y + enemy.get_height() > HEIGHT :
                lives -=1
                enemies.remove(enemy)
                
            

        player.move_lasers(-laser_vel , enemies)
        
            
main()        